In [1]:
import ccxt
import pandas as pd
import numpy as np
import talib

# Initialize Binance
exchange = ccxt.binance()

timeframe = '1h'  # hourly candles
limit = 10000000  # number of candles to fetch




In [2]:
import ccxt
import pandas as pd
import time

# Initialize exchange
exchange = ccxt.binance()

symbol = 'BTC/USDT'
timeframe = '15m'   # 15-minute candles
limit_per_request = 1000
total_limit = 10000

def fetch_all_ohlcv(symbol, timeframe, total_limit=10000, limit_per_request=1000, pause=1):
    all_data = []
    # Start from the latest available timestamp
    since = None

    while len(all_data) < total_limit:
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since=since, limit=limit_per_request)

        if not ohlcv:
            break

        # If first batch, just add it
        if not all_data:
            all_data = ohlcv
        else:
            # Append older data to the *beginning* (we’re going backward)
            all_data = ohlcv + all_data

        print(f"Fetched {len(all_data)} candles so far...")

        # Move `since` backward (1000 candles earlier)
        # Each 15m candle = 15 * 60 * 1000 ms = 900000 ms
        since = ohlcv[0][0] - (limit_per_request * 15 * 60 * 1000)

        time.sleep(pause)

        # Stop if we can’t get more data
        if len(ohlcv) < limit_per_request:
            break

    # Convert to DataFrame
    df = pd.DataFrame(all_data, columns=['timestamp','open','high','low','close','volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    df = df.drop_duplicates()
    return df

# Fetch
df = fetch_all_ohlcv(symbol, timeframe, total_limit=10000)

print("Final shape:", df.shape)
print(df.head(), "\n")
print(df.tail())


Fetched 1000 candles so far...
Fetched 2000 candles so far...
Fetched 3000 candles so far...
Fetched 4000 candles so far...
Fetched 5000 candles so far...
Fetched 6000 candles so far...
Fetched 7000 candles so far...
Fetched 8000 candles so far...
Fetched 9000 candles so far...
Fetched 10000 candles so far...
Final shape: (10000, 5)
                          open       high        low      close     volume
timestamp                                                                 
2025-07-29 04:30:00  118653.16  118685.00  118502.05  118514.87  111.34730
2025-07-29 04:45:00  118514.88  118568.95  118416.66  118533.93   84.03811
2025-07-29 05:00:00  118533.93  118753.20  118527.77  118666.23   75.89476
2025-07-29 05:15:00  118666.22  118666.23  118540.49  118649.26   95.06615
2025-07-29 05:30:00  118649.27  118748.77  118597.78  118746.94  135.28061 

                          open       high        low      close     volume
timestamp                                                      

In [3]:
df.head()

,open,high,low,close,volume
timestamp,,,,,
2025-07-29 04:30:00,118653.16,118685.00,118502.05,118514.87,111.34730
2025-07-29 04:45:00,118514.88,118568.95,118416.66,118533.93,84.03811
2025-07-29 05:00:00,118533.93,118753.20,118527.77,118666.23,75.89476
2025-07-29 05:15:00,118666.22,118666.23,118540.49,118649.26,95.06615
2025-07-29 05:30:00,118649.27,118748.77,118597.78,118746.94,135.28061


In [4]:
df.shape

(10000, 5)

In [5]:
"""# ===============================
# 2. Calculate RSI
# ===============================
import ta
df["RSI"] = ta.momentum.RSIIndicator(df["close"], window=14).rsi()

# ===============================
# 3. MACD Calculation
# ===============================
df["EMA12"] = df["close"].ewm(span=12, adjust=False).mean()
df["EMA26"] = df["close"].ewm(span=26, adjust=False).mean()
df["MACD"] = df["EMA12"] - df["EMA26"]
df["Signal"] = df["MACD"].ewm(span=9, adjust=False).mean()
df["Histogram"] = df["MACD"] - df["Signal"]

import talib
df["DEMA9"] = talib.DEMA(df["close"].values, timeperiod=9)

# ------------------------------
# 2. Calculate SMA (3 closes)
# ------------------------------
sma_window = 3
df['SMA'] = ta.trend.sma_indicator(df['close'], window=sma_window)


"""

'# ===============================\n# 2. Calculate RSI\n# ===============================\nimport ta\ndf["RSI"] = ta.momentum.RSIIndicator(df["close"], window=14).rsi()\n\n# ===============================\n# 3. MACD Calculation\n# ===============================\ndf["EMA12"] = df["close"].ewm(span=12, adjust=False).mean()\ndf["EMA26"] = df["close"].ewm(span=26, adjust=False).mean()\ndf["MACD"] = df["EMA12"] - df["EMA26"]\ndf["Signal"] = df["MACD"].ewm(span=9, adjust=False).mean()\ndf["Histogram"] = df["MACD"] - df["Signal"]\n\nimport talib\ndf["DEMA9"] = talib.DEMA(df["close"].values, timeperiod=9)\n\n# ------------------------------\n# 2. Calculate SMA (3 closes)\n# ------------------------------\nsma_window = 3\ndf[\'SMA\'] = ta.trend.sma_indicator(df[\'close\'], window=sma_window)\n\n\n'

In [6]:
"""# ------------------------------
# 2. Compute TSI
# ------------------------------
def compute_tsi(close, r1=25, r2=13):
    delta = close.diff()
    ema1 = delta.ewm(span=r1, adjust=False).mean()
    ema2 = ema1.ewm(span=r2, adjust=False).mean()

    abs_delta = delta.abs()
    abs_ema1 = abs_delta.ewm(span=r1, adjust=False).mean()
    abs_ema2 = abs_ema1.ewm(span=r2, adjust=False).mean()

    tsi = 100 * (ema2 / abs_ema2)
    return tsi

df['TSI'] = compute_tsi(df['close'])
"""

"# ------------------------------\n# 2. Compute TSI\n# ------------------------------\ndef compute_tsi(close, r1=25, r2=13):\n    delta = close.diff()\n    ema1 = delta.ewm(span=r1, adjust=False).mean()\n    ema2 = ema1.ewm(span=r2, adjust=False).mean()\n\n    abs_delta = delta.abs()\n    abs_ema1 = abs_delta.ewm(span=r1, adjust=False).mean()\n    abs_ema2 = abs_ema1.ewm(span=r2, adjust=False).mean()\n\n    tsi = 100 * (ema2 / abs_ema2)\n    return tsi\n\ndf['TSI'] = compute_tsi(df['close'])\n"

In [7]:
import ta  # pip install ta
import talib  # optional but used earlier; if missing remove DEMA or use pandas ewm

def add_features(df):
    df = df.copy()
    # ensure columns lower-case/consistent
    # expect df to contain 'open','high','low','close','volume' (case-insensitive)
    df.columns = [c.lower() for c in df.columns]

    # basic indicators (your current set)
    df['rsi'] = ta.momentum.RSIIndicator(df['close'], window=14).rsi()
    df['ema12'] = df['close'].ewm(span=12, adjust=False).mean()
    df['ema26'] = df['close'].ewm(span=26, adjust=False).mean()
    df['macd'] = df['ema12'] - df['ema26']
    df['signal'] = df['macd'].ewm(span=9, adjust=False).mean()
    df['histogram'] = df['macd'] - df['signal']

    # DEMA (if talib available)
    try:
        df['dema9'] = talib.DEMA(df['close'].values, timeperiod=9)
    except Exception:
        df['dema9'] = df['close'].ewm(span=9).mean()  # fallback

    df['sma3'] = ta.trend.sma_indicator(df['close'], window=3)

    # TSI
    def compute_tsi(close, r1=25, r2=13):
        delta = close.diff()
        ema1 = delta.ewm(span=r1, adjust=False).mean()
        ema2 = ema1.ewm(span=r2, adjust=False).mean()
        abs_delta = delta.abs()
        abs_ema1 = abs_delta.ewm(span=r1, adjust=False).mean()
        abs_ema2 = abs_ema1.ewm(span=r2, adjust=False).mean()
        tsi = 100 * (ema2 / (abs_ema2.replace(0, np.nan)))
        return tsi.fillna(0)
    df['tsi'] = compute_tsi(df['close'])

    # Stochastic %K and %D
    period = 14; smooth_k = 3; smooth_d = 3
    lowest_low = df['low'].rolling(period).min()
    highest_high = df['high'].rolling(period).max()
    df['%k'] = 100 * (df['close'] - lowest_low) / (highest_high - lowest_low + 1e-8)
    df['%k'] = df['%k'].rolling(smooth_k).mean()
    df['%d'] = df['%k'].rolling(smooth_d).mean()

    # --- NEW HIGH-IMPACT features ---
    # ATR (volatility)
    df['atr'] = ta.volatility.AverageTrueRange(df['high'], df['low'], df['close'], window=14).average_true_range()
    # On-Balance Volume
    df['obv'] = ta.volume.OnBalanceVolumeIndicator(df['close'], df['volume']).on_balance_volume()
    # ADX (trend strength)
    df['adx'] = ta.trend.ADXIndicator(df['high'], df['low'], df['close'], window=14).adx()
    # CCI
    df['cci'] = ta.trend.cci(df['high'], df['low'], df['close'], window=20)

    # Price-derived features
    df['return_1'] = df['close'].pct_change()
    df['roll_mean_5'] = df['close'].rolling(5).mean()
    df['roll_std_5'] = df['close'].rolling(5).std()

    # drop rows with NaNs resulting from indicators
    df = df.dropna().reset_index(drop=True)
    return df
df = add_features(df)

In [8]:
"""# ========================
# 2. Stochastic Oscillator
# ========================
period = 14
smooth_k = 3
smooth_d = 3

lowest_low = df["low"].rolling(period).min()
highest_high = df["high"].rolling(period).max()

df["%K"] = 100 * (df["close"] - lowest_low) / (highest_high - lowest_low)
df["%K"] = df["%K"].rolling(smooth_k).mean()
df["%D"] = df["%K"].rolling(smooth_d).mean()
"""

'# ========================\n# 2. Stochastic Oscillator\n# ========================\nperiod = 14\nsmooth_k = 3\nsmooth_d = 3\n\nlowest_low = df["low"].rolling(period).min()\nhighest_high = df["high"].rolling(period).max()\n\ndf["%K"] = 100 * (df["close"] - lowest_low) / (highest_high - lowest_low)\ndf["%K"] = df["%K"].rolling(smooth_k).mean()\ndf["%D"] = df["%K"].rolling(smooth_d).mean()\n'

In [9]:
df.head()

,open,high,low,close,volume,rsi,ema12,ema26,macd,signal,...,tsi,%k,%d,atr,obv,adx,cci,return_1,roll_mean_5,roll_std_5
0,118818.23,118948.02,118758.98,118924.82,54.94954,57.272355,118868.479741,118783.329680,85.150061,89.578475,...,29.454178,33.291106,40.677275,192.577674,441.86654,0.0,37.056360,0.000897,118887.044,75.558858
1,118924.82,118924.83,118785.75,118913.07,118.54983,56.612631,118875.339781,118792.940074,82.399707,88.142722,...,27.383025,27.553060,33.954628,188.756412,323.31671,0.0,30.160083,-0.000099,118910.316,56.226674
2,118913.07,118913.07,118761.22,118784.79,116.19715,49.859969,118861.409045,118792.336365,69.072680,84.328713,...,23.284667,24.394752,28.412973,186.120240,207.11956,0.0,-27.035946,-0.001079,118872.940,66.433034
3,118784.80,118828.01,118560.00,118586.80,215.24550,41.610380,118819.161500,118777.111449,42.050051,75.872981,...,16.617961,13.764172,21.903995,191.969508,-8.12594,0.0,-178.546438,-0.001667,118805.540,136.224953
4,118586.79,118708.51,118543.11,118659.32,166.13026,45.187698,118794.570500,118768.386157,26.184343,65.935253,...,12.598890,8.720862,15.626595,190.071686,158.00432,0.0,-202.241529,0.000612,118773.760,150.331813


In [10]:
df.shape

(9981, 23)

In [11]:
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)

In [12]:
df.shape

(9981, 23)

In [13]:
"""import numpy as np

def adaptive_triple_barrier_labeling(df, mult=2, max_period=30):
    ""
    Adaptive triple-barrier method based on volatility.
    - mult: how many standard deviations to use for barrier width
    - max_period: candles to look ahead
    ""
    df = df.copy()
    df['vol'] = df['close'].pct_change().rolling(20).std()
    labels = np.ones(len(df), dtype=int)
    close = df['close'].values

    for i in range(len(df) - max_period):
        if np.isnan(df['vol'].iloc[i]):
            continue

        # Dynamic upper/lower barriers based on volatility
        upper = close[i] * (1 + mult * df['vol'].iloc[i])
        lower = close[i] * (1 - mult * df['vol'].iloc[i])
        future_prices = close[i+1:i+max_period+1]

        if np.any(future_prices >= upper):
            labels[i] = 2
        elif np.any(future_prices <= lower):
            labels[i] = 0
        else:
            labels[i] = 1

    df['Label'] = labels
    return df
"""

'import numpy as np\n\ndef adaptive_triple_barrier_labeling(df, mult=2, max_period=30):\n    ""\n    Adaptive triple-barrier method based on volatility.\n    - mult: how many standard deviations to use for barrier width\n    - max_period: candles to look ahead\n    ""\n    df = df.copy()\n    df[\'vol\'] = df[\'close\'].pct_change().rolling(20).std()\n    labels = np.ones(len(df), dtype=int)\n    close = df[\'close\'].values\n\n    for i in range(len(df) - max_period):\n        if np.isnan(df[\'vol\'].iloc[i]):\n            continue\n\n        # Dynamic upper/lower barriers based on volatility\n        upper = close[i] * (1 + mult * df[\'vol\'].iloc[i])\n        lower = close[i] * (1 - mult * df[\'vol\'].iloc[i])\n        future_prices = close[i+1:i+max_period+1]\n\n        if np.any(future_prices >= upper):\n            labels[i] = 2\n        elif np.any(future_prices <= lower):\n            labels[i] = 0\n        else:\n            labels[i] = 1\n\n    df[\'Label\'] = labels\n    ret

In [14]:
"""df = adaptive_triple_barrier_labeling(df, mult=2, max_period=30)
print(df["Label"].value_counts())
"""

'df = adaptive_triple_barrier_labeling(df, mult=2, max_period=30)\nprint(df["Label"].value_counts())\n'

In [15]:
"""# ========================
# 3. Swing Highs / Lows
# ========================
lookback_swings = 5

def find_swings(prices, lb):
    labels = [0]*len(prices)
    for i in range(lb, len(prices)-lb):
        if prices[i] == max(prices[i-lb:i+lb+1]):
            labels[i] = -1   # swing high
        elif prices[i] == min(prices[i-lb:i+lb+1]):
            labels[i] = 1    # swing low
    return labels

df["Label"] = find_swings(df["close"].values, lookback_swings)"""


'# ========================\n# 3. Swing Highs / Lows\n# ========================\nlookback_swings = 5\n\ndef find_swings(prices, lb):\n    labels = [0]*len(prices)\n    for i in range(lb, len(prices)-lb):\n        if prices[i] == max(prices[i-lb:i+lb+1]):\n            labels[i] = -1   # swing high\n        elif prices[i] == min(prices[i-lb:i+lb+1]):\n            labels[i] = 1    # swing low\n    return labels\n\ndf["Label"] = find_swings(df["close"].values, lookback_swings)'

In [16]:
"""df['Label'] = df['Label'] + 1"""

"df['Label'] = df['Label'] + 1"

In [17]:
import pandas as pd
import numpy as np

def label_reversals(df, window=5, atr_period=14, atr_mult=1.5):
    """
    Label reversal points using Swing High/Low + ATR filter.
    
    Parameters
    ----------
    df : pd.DataFrame
        Must contain ['High', 'Low', 'Close'] columns.
    window : int
        Number of candles on each side for swing detection.
    atr_period : int
        Period for ATR calculation.
    atr_mult : float
        Minimum price movement (in multiples of ATR) to qualify as a true reversal.
    
    Returns
    -------
    df : pd.DataFrame
        Original dataframe with a new column 'Label':
          0 = Top reversal
          1 = No reversal
          2 = Bottom reversal
    """

    df = df.copy().reset_index(drop=True)  # ✅ Ensure numeric index

    # --- Calculate ATR (Average True Range) ---
    high_low = df['high'] - df['low']
    high_close = np.abs(df['high'] - df['close'].shift())
    low_close = np.abs(df['low'] - df['close'].shift())
    true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    df['ATR'] = true_range.rolling(atr_period).mean()

    # --- Detect potential swing highs/lows ---
    df['swing_high'] = df['high'][(df['high'] == df['high'].rolling(window*2+1, center=True).max())]
    df['swing_low']  = df['low'][(df['low'] == df['low'].rolling(window*2+1, center=True).min())]

    # --- Filter out minor reversals using ATR threshold ---
    df['Label'] = 1  # default: no reversal

    for i in range(window, len(df) - window):
        atr = df.loc[i, 'ATR']
        if np.isnan(atr):
            continue

        # Check for swing high
        if not np.isnan(df.loc[i, 'swing_high']):
            # Price must have fallen more than atr_mult * ATR afterwards
            future_min = df['low'].iloc[i:i+window].min()
            if df.loc[i, 'high'] - future_min > atr_mult * atr:
                df.loc[i, 'Label'] = 0  # Top reversal

        # Check for swing low
        if not np.isnan(df.loc[i, 'swing_low']):
            # Price must have risen more than atr_mult * ATR afterwards
            future_max = df['high'].iloc[i:i+window].max()
            if future_max - df.loc[i, 'low'] > atr_mult * atr:
                df.loc[i, 'Label'] = 2  # Bottom reversal

    # --- Clean up temporary columns ---
    df.drop(['swing_high', 'swing_low'], axis=1, inplace=True)
    
    return df


In [18]:
df = label_reversals(df, window=5, atr_period=14, atr_mult=1.5)

print(df['Label'].value_counts())

Label
1    8921
2     535
0     525
Name: count, dtype: int64


In [19]:
df.head()

,open,high,low,close,volume,rsi,ema12,ema26,macd,signal,...,%d,atr,obv,adx,cci,return_1,roll_mean_5,roll_std_5,ATR,Label
0,118818.23,118948.02,118758.98,118924.82,54.94954,57.272355,118868.479741,118783.329680,85.150061,89.578475,...,40.677275,192.577674,441.86654,0.0,37.056360,0.000897,118887.044,75.558858,NaN,1
1,118924.82,118924.83,118785.75,118913.07,118.54983,56.612631,118875.339781,118792.940074,82.399707,88.142722,...,33.954628,188.756412,323.31671,0.0,30.160083,-0.000099,118910.316,56.226674,NaN,1
2,118913.07,118913.07,118761.22,118784.79,116.19715,49.859969,118861.409045,118792.336365,69.072680,84.328713,...,28.412973,186.120240,207.11956,0.0,-27.035946,-0.001079,118872.940,66.433034,NaN,1
3,118784.80,118828.01,118560.00,118586.80,215.24550,41.610380,118819.161500,118777.111449,42.050051,75.872981,...,21.903995,191.969508,-8.12594,0.0,-178.546438,-0.001667,118805.540,136.224953,NaN,1
4,118586.79,118708.51,118543.11,118659.32,166.13026,45.187698,118794.570500,118768.386157,26.184343,65.935253,...,15.626595,190.071686,158.00432,0.0,-202.241529,0.000612,118773.760,150.331813,NaN,1


In [20]:
df.to_csv('btc_usdt8.csv', index=False)